<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Upper Type Bounds

In Scala, [type variables](generic-classes.ipynb) may be constrained by a type bound. 

- Type bounds limit the concrete values of the type variables.
  
- An _upper type bound_ or is a special case of _type constraint_, where **`T <: A` declares that `T` is subtype of `A`.**

**Example:**

- Suppose we have some herarchical relationship between types for animals and pets.

    ```scala
    // Animals
    abstract class Animal:
        def name: String
    
    class Lion extends Animal:
      override def name: String = "Lion"
    
    // Pets
    abstract class Pet extends Animal
    
    class Cat extends Pet:
      override def name: String = "Cat"
    
    class Dog extends Pet:
      override def name: String = "Dog"

    ```

    <br/>

- And we define some instances of those classes:

  ```scala
    // Defining some animals...
    val lion = new Lion
    val dog = new Dog
    val cat = new Cat
    
    ```

- An we also define some generic containers. Some of them constrained by type.

    ```scala
    // Containers
    class Container[A](x: A): // generic type variable, can be anything
        def elem: A = x
    
    class AnimalContainer[A <: Animal](x: A): // notice the type constraint
        def elem: A = x
    
    class PetContainer[A <: Pet](x: A): // notice the type constraint
        def elem: A = x

    ```

Questions... 🖐️

- _Can I add the `lion` in `Container[Animal]`, why?_

    ```scala
    Container[Animal](lion)
    
    ```

    <br/>

- _Can I add the `lion` in `PetContainer[Lion]`, why?_

    ```scala
    PetContainer[Lion](lion)

    ```

    <br/>

- _Can I add the `lion` and the `dog` in `AnimalContainer[Animal]`, why?_

    ```scala
    AnimalContainer[Animal](lion)
    AnimalContainer[Animal](dog)

    ```

    <br/>

- _Can I add the `dog` and the `cat` in `PetContainer[Cat]`, why?_

    ```scala
    PetContainer[Cat](dog)
    PetContainer[Cat](cat)

    ```


## Constraining supported types

**Some generic types and methods only make sense when the type parameter supports certain behavior/s:**

```scala
import scala.language.postfixOps


trait Comparable[A]:
    def compare(that: A): Int        // Abstract method to be implemented...
    def < (that: A): Boolean = (this `compare` that) < 0
    def > (that: A): Boolean = (this `compare` that) > 0
    def <= (that: A): Boolean = (this `compare` that) <= 0
    def >= (that: A): Boolean = (this `compare` that) >= 0
    def compareTo(that: A): Int = compare(that)

abstract class Set[A <: Comparable[A]]:
    def incl(x: A): Set[A]
    def contains(x: A): Boolean

class EmptySet[A <: Comparable[A]] extends Set[A]:
    def contains(x: A): Boolean = false
    def incl(x: A): Set[A] = new NonEmptySet(x, new EmptySet[A], new EmptySet[A])

class NonEmptySet[A <: Comparable[A]](e: A, lset: Set[A], rset: Set[A]) extends Set[A]:
    def contains(x: A): Boolean =
        if (x < e) lset `contains` x // T needs to be comparable
        else if (x > e) rset `contains` x
        else true
    def incl(x: A): Set[A] =
        if (x < e) new NonEmptySet(e, lset `incl` x, rset)
        else if (x > e) new NonEmptySet(e, lset, rset `incl` x)
        else this

```
- For a set `Set[A]` to work correctly:
  
    - Elements of type `A` must be comparable.
      
    - We capture this requirement by stating that `A` is a subtype of `Comparable[A]`.
 
    - We enforce this requirement by adding the type constraint `[A <: Comparable[A]]` in the definition of `Set`.

- Capturing subtyping between `Num` and `Comparable[Num]`:

    ```scala
    case class Num(value: Double) extends Comparable[Num]:
        def compare(that: Num): Int =
            if (this.value < that.value) -1
            else if (this.value > that.value) 1
            else 0
    
    val s = new EmptySet[Num].incl(Num(9)).incl(Num(0)).incl(Num(10))
    
    ```

    <br/>

    - <span style="color:red">**We are required only to implement the `compare` method as it is abstract in trait `Comparable`.**</span>

<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>